In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import czifile as zis
from apeer_ometiff_library import io, processing, omexmlClass
import os
from matplotlib import pyplot as plt, cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from skimage.viewer import ImageViewer
import skimage.io
import matplotlib.colors as colors
import numpy as np
#from skimage.external import tifffile
import ipywidgets as widgets

import imgfileutils as imf

In [2]:
basefolder = r'/datadisk1/tuxedo/IPython_Notebooks/testdata'
#basefolder = r'/home/sebi06/testdata'

#filename = os.path.join(basefolder, 'Filter_with_Particles_big.ome.tiff')
#filename = os.path.join(basefolder, 'S=2_T=5_CH=3_CH=2_A2.ome.tiff')
#filename = os.path.join(basefolder, 'Osteosarcoma_01.ome.tiff')
filename = os.path.join(basefolder, 'Filter_with_Particles_small.czi')
#filename = os.path.join(basefolder, '8Brains_DAPI_5X_stitched.czi')
#filename = os.path.join(basefolder, 'S=2_T=5_Z=3_CH=2_A2.czi')

In [4]:
image_name = os.path.basename(filename)

if filename.lower().endswith('.ome.tiff') or filename.lower().endswith('.ome.tif'):
    
    # Return value is an 5D Array of order (T, Z, C, X, Y)
    (array, omexml) = io.read_ometiff(filename)
    metadata = imf.get_metadata(filename, series=0)
    
if filename.lower().endswith('.czi'):
    
    metadata = imf.get_metadata(filename)
    
    print(metadata['Shape'])
    print(metadata['Axes'])

    
    array = imf.get_array_czi(filename,
                              cziaxes=metadata['Axes'],
                              blockindex=0,
                              sceneindex=0,
                              replacezero=True)
    print(array.shape)

Image Type:  czi
Getting CZI Metadata ...
(1, 1, 1, 2061, 4662, 1)
TCZYX0
{'B': -1, 'S': -1, 'T': 0, 'C': 1, 'Z': 2, 'Y': 3, 'X': 4, '0': 5}
[-1, -1, 0, 1, 2, 3, 4, 5]
5
cut 5
(1, 1, 1, 2061, 4662)


In [5]:
print(metadata['Shape'])
print(metadata['Axes'])
print(array.shape)

(1, 1, 1, 2061, 4662, 1)
TCZYX0
(1, 1, 1, 2061, 4662)


In [6]:
#out = widgets.interactive_output(get_TZC, { 't': t, 'z': z, 'c': c, 'r':r})
ui, out = imf.create_ipyviewer(array, metadata)

display(ui, out)

Output()

In [ ]:
def get_dimorder(dimstring):
    
    dimindex_list = []
    dims = ['B', 'S', 'T', 'C', 'Z', 'Y', 'X', '0']
    dims_dict = {}
    
    for d in dims:
    
        dims_dict[d] = dimstring.find(d)
        dimindex_list.append(dimstring.find(d))
        
    numvalid_dims = sum(i > 0 for i in dimindex_list)
    
    return dims_dict, dimindex_list, numvalid_dims

dim_dict, dim_list, numvalid_dims = get_dimorder(metadata['Axes'])

print(dim_dict)
print(dim_list)
print(numvalid_dims)

In [ ]:
if dim_dict['B'] < 0 and dim_dict['S'] >=0:
    # B does not exist but S does
    cut = (0, count)
if dim_dict['B'] >= 0 and dim_dict['S'] >=0:
    # B and S do exist
    cut = (0, 1, count)
if dim_dict['B'] < 0 and dim_dict['S'] < 0:
    # B and S do not exist
    cut = (count)
if dim_dict['B'] >= 0 and dim_dict['S'] <0:
    # B does exist but S does not
    cut = (0, count)
    
print(cut)

In [ ]:
out = np.squeeze(array, axis=(count))
print(array.shape)
print(out.shape)